# TP sur le topic modeling (corrigé)

François HU - Data scientist au DataLab de la Société Générale Assurances - *13/10/20* - https://nbviewer.jupyter.org/github/curiousML/DSA/tree/master/text_mining/

In [3]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

**Introduction**

Les topic models (*modèles thématiques* en français) sont une famille de modèles qui sont capables de découvrir les topics (*thèmes* en français) d'une collection de documents textuels. Dans ce contexte, le terme "**topic**" désigne des groupes de mots qui se retrouvent souvent ensemble dans un même document. Par exemple, dans un recueil d'articles de journaux, un topic model peut identifier un topic composé des mots :
- "homme politique", "droit" et "parlement", et un autre caractérisé par 
- "joueur", "match" et "carton rouge"

Les topics modeling ne peuvent pas affecter un titre à ces topics : c'est notre tâche d'interpréter ces topics et de leur donner des étiquettes telles que "politique" et "football".

L'un des modèles les plus populaires est le LDA. Le LDA (cf. <font color = "red">sections précédentes</font>) est un modèle génératif qui considère chaque document comme un mélange de topics. Ce sont ces topics qui seront en charge de générer les mots. Par exemple, le topic "football" générera le mot "pénalité" avec une probabilité élevée, tandis que le topic "politique" aura une probabilité beaucoup plus élevée pour générer le mot "politicien" que pour générer le mot "pénalité".

### Données : Grand Débat National

- L'un des contextes où la modélisation des topics est très utile est celui des questions ouvertes. Il nous permet d'explorer la variation des topics abordés dans les réponses des gens. 

- Dans ce notebook nous allons explorer un ensemble complet de réponses du [Grand Débat National](https://granddebat.fr/), le débat public organisé par le président Macron. Le but du débat était de mieux comprendre les besoins et les opinions des Français suite aux manifestations des gilets jaunes. Les résultats de ce débat sont maintenant disponibles sous forme de [données ouvertes](https://granddebat.fr/pages/donnees-ouvertes).

### 3.1. Charger les données

Nous allons tout d'abord télécharger un des fichiers csv sur la transition écologique et charger le contenu dans un dataframe [pandas](https://pandas.pydata.org/)

In [6]:
import pandas as pd

# indiquer le chemin du fichier
chemin = "https://raw.githubusercontent.com/curiousML/DSA/master/text_mining/data/LA_TRANSITION_ECOLOGIQUE.csv"
#"/chemin/menant/a/LA_TRANSITION_ECOLOGIQUE.csv"

# notre dataframe
df = pd.read_csv(chemin, error_bad_lines=False, warn_bad_lines=False)

/Users/Faugon/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chacune des lignes de ce dataframe ``df`` contient des réponses répondant à une liste de questions sur la transition écologique. Certaines de ces questions sont à choix multiples, tandis que d'autres sont des questions ouvertes. 

In [7]:
df.columns

Index(['reference', 'title', 'createdAt', 'publishedAt', 'updatedAt',
       'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode',
       'Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?',
       'Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?',
       'Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?',
       'Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?',
       'À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?',
       'Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?',
       'Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?',
   

Nous nous focaliserons sur la dernière question : ``Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?`` car elle donne le plus de liberté aux personnes.

Nous espérons que notre modèle LDA nous aidera à analyser les topics sur lesquels portent leurs réponses.

In [3]:
question = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
df[question].head(10)

0               Multiplier les centrales géothermiques
1    Les problèmes auxquels se trouve confronté l’e...
2                                                  NaN
3                                                  NaN
4      Une vrai politique écologique et non économique
5    Les bonnes idées ne grandissent que par le par...
6    Pédagogie dans ce sens là dés la petite école ...
7                                                  NaN
8    faire de l'écologie incitative et non punitive...
9    Développer le ferroutage pour les poids lourds...
Name: Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?, dtype: object

Comme nous le remarquons, il y a beaucoup de données manquantes (comme toute question ouverte, les personnes décident oui ou non d'écrire un commentaire). Une étape de néttoyage est nécessaire.

### 3.2. Nettoyer et vectoriser les documents

Avant d'entraîner notre modèle LDA, nous avons besoin de tokenizer notre texte. Nous allons tokenizer grâce à la librarie [spaCy](https://spacy.io/) car nous allons effectuer seulement quelques prétraitements de base (pour les personnes intéressées, des p). Nous allons juste initialiser un modèle vierge pour la langue française.

In [4]:
import spacy

nlp = spacy.blank("fr")

Supprimons toutes les lignes du dataframe qui n'ont pas de réponse pour notre question (les `NaN`s ci-dessus). Ce nouveau dataframe s'appellera ``textes``

In [5]:
textes = df[df[question].notnull()][question] # A REMPLIR

Ensuite, nous utilisons spaCy pour effectuer notre premier prétraitement :

In [6]:
%time spacy_docs = list(nlp.pipe(textes))

CPU times: user 14.7 s, sys: 214 ms, total: 14.9 s
Wall time: 14.9 s


Nous avons maintenant une liste de documents spaCy. Nous allons transformer chaque document spaCy en une liste de tokens. Au lieu des tokens originaux, nous allons travailler avec les lemmes à la place. Cela permettra à notre modèle de mieux généraliser, car il pourra "voir" que "géothermiques" et "géothermique" représentent la même signification. Voici la liste complète des prétraitements : 
 
- supprimer tous les **mots de moins de 3 caractères**,
- supprimer tous les **stop-words**, et
- **lemmatiser** les mots restants et,
- mettre ces mots en **minuscules**.

In [7]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 3 and not t.is_stop] for doc in spacy_docs]
print(docs[:3])

[['multiplier', 'centrales', 'géothermiques'], ['problèmes', 'trouve', 'confronté', 'ensemble', 'planète', 'dénoncent', 'parfait', 'désordre', 'gilets', 'jaunes', 'france', 'surpopulation', 'mondiale', 'population', 'passée', 'd’1,5', 'milliards', 'habitants', '1900', 'milliards', '2020', 'montera', 'bientôt', 'milliards', '2040', 'progrès', 'communication', 'village', 'mondial', 'individu', 'fond', 'asie', 'fond', 'afrique', 'passant', 'quartiers', 'campagnes', 'pays', 'aspire', 'vivre', 'blâmer', 'lotis', 'concitoyens', 'logement', 'nourriture', 'biens', 'consommation', 'déplacement', 'mère', 'problèmes', 'solution', 'problèmes', 'stabilisation', 'croissance', 'démographique', 'partage', 'richesses', 'partage', 'terres', 'partage', 'protection', 'biodiversité', 'règlement', 'conflits', 'lutte', 'déforestation', 'lutte', 'dérèglement', 'climatique', 'règlement', 'conflits', 'stabilisation', 'migrations', 'concurrence', 'commerciale', 'mondiale', 'etc.', 'française', 'européenne', 'mon

Afin de conserver un peu les ordres des mots dans notre modélisation, nous allons tenir en compte les bigrammes fréquents. Pour cela, nous allons utiliser la bibliothèque [Gensim](https://radimrehurek.com/gensim/). Nous tenons à remarquer que la bibliothèse Gensim est une excellente bibliothèque NLP pour les topics modeling. 

Voici le processus retenu : 

- Nous identifions d'abord les bigrammes fréquents dans le corpus, 
- puis nous les ajoutons à la liste des tokens pour les documents dans lesquels ils apparaissent. Cela signifie que les bigrammes ne seront pas dans leur position correcte dans le texte, mais cela ne pose pas de soucis : les topic models sont des modèles de bag-of-words (*sac-de-mots* en français) qui ignorent la position des mots de toute façon.

In [8]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            # A REMPLIR (DEBUT)
            docs[idx].append(token)
            # (FIN)

In [9]:
docs[2]

['vrai', 'politique', 'écologique', 'économique']

Passons aux dernières étapes du prétraitement spécifique à Gensim. Nous allons tout d'abord créer une représentation dictionnaire des documents. Ce dictionnaire mappera chaque mot à un identifiant unique et nous aidera à créer des représentations en sac-de-mot de chaque document. Ces représentations en sac-de-mots contiennent les identificateurs des mots du document ainsi que leur fréquence. De plus, nous pouvons supprimer les mots les moins fréquents et les plus fréquents du vocabulaire. Cela améliorera la qualité de notre modèle et accélèrera son entraînement. La fréquence minimale d'un mot est exprimée en nombre absolu, la fréquence maximale est la proportion de documents dans lesquels un mot peut figurer.

In [10]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents :', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Number of unique words after removing rare and common words :', len(dictionary))

print("Example representation of document 3 :", dictionary.doc2bow(docs[2]))

Number of unique words in original documents : 43652
Number of unique words after removing rare and common words : 17759
Example representation of document 3 : [(85, 1), (86, 1), (87, 1), (88, 1)]


Ensuite, nous créons des représentations en sac de mots pour chaque document du corpus voir la méthode [doc2bow](https://radimrehurek.com/gensim/corpora/dictionary.html) :

In [11]:
corpus = [ dictionary.doc2bow(doc) for doc in docs] # A REMPLIR

### 3.3. Topic Modeling avec LDA

Maintenant, il est temps d'entraîner notre LDA ! Pour ce faire, nous utilisons les paramètres suivants : 

- **corpus** : les représentations en sac-de-mots de nos documents
- **id2token** : le mappage des index aux mots
- **num_topics** : le nombre de topics que le modèle doit identifier (fixons à <font color = "red"><b>10</b></font>)
- **chunksize** : le nombre de documents que le modèle voit à chaque mise à jour (fixons à <font color = "red"><b>1 000</b></font>)
- **passes** : le nombre de fois où nous montrons le corpus total au modèle pendant l'entraînement (fixons à <font color = "red"><b>5</b></font>)
- **random_state** : nous utilisons une graine pour assurer la reproductibilité (fixons à <font color = "red"><b>1</b></font>)

Sur un corpus de cette taille, l'entraînement dure généralement une ou deux minutes.

In [12]:
from gensim.models import LdaModel

%time model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1) # A REMPLIR

CPU times: user 37.4 s, sys: 185 ms, total: 37.5 s
Wall time: 36.6 s


### 3.3. Résultats et visualisation

Voyons ce que le modèle a appris. Pour ce faire, affichons les dix mots les plus caractéristiques pour chacun des topics. Nous obeservons déjà des tendances intéressantes : si certains topics sont plus généraux (comme le topic 3), d'autres font référence à des topics très pertinents : 
- véhicules électriques (**topic 1**), 
- énergie (alternative) (**topic 2**), 
- agriculture (**topic 6**), 
- déchets et recyclage (**topic 7**) et 
- fiscalité (**topic 9**).

In [13]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.052*"produits" + 0.021*"agriculture" + 0.017*"interdire" + 0.014*"animaux" + 0.012*"pesticides" + 0.011*"agriculteurs" + 0.010*"production" + 0.009*"consommation" + 0.009*"santé" + 0.008*"favoriser"

***********
* topic 2 *
***********
2 : 0.010*"climatique" + 0.007*"problème" + 0.007*"planète" + 0.006*"population" + 0.006*"réchauffement" + 0.005*"années" + 0.005*"grand" + 0.005*"changement" + 0.004*"problèmes" + 0.004*"croissance"

***********
* topic 3 *
***********
3 : 0.024*"protection" + 0.018*"environnement" + 0.017*"pollueur" + 0.015*"grande_distribution" + 0.013*"distribution" + 0.013*"payeur" + 0.011*"arrêtons" + 0.011*"principe" + 0.010*"obsolescence" + 0.009*"grande"

***********
* topic 4 *
***********
4 : 0.012*"villes" + 0.012*"déchets" + 0.012*"entreprises" + 0.011*"grandes" + 0.009*"zones" + 0.006*"ville" + 0.006*"obliger" + 0.006*"faut" + 0.006*"faire" + 0.006*"centre"

***********
* topic 5 *
***********
5 : 0.035*"vitesse" + 

Une autre façon d'observer les topics est de les **visualiser**. Ceci peut être fait avec la bibliothèque [pyLDAvis](https://github.com/bmabey/pyLDAvis). PyLDAvis nous montrera à quel point les sujets sont populaires dans notre corpus, à quel point les sujets sont similaires et quels sont les mots les plus importants pour ce sujet. Notez qu'il est important de définir ``sort_topics = False`` sur l'appel à pyLDAvis. Si vous ne le faites pas, les sujets seront classés différemment de ceux de Gensim. Cela peut prendre quelques minutes pour charger.

In [14]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

/miniconda3/envs/xeus-cling2/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.065714 -0.212035       1        1  10.444253
1      0.029440  0.209894       2        1   8.859770
2      0.271206 -0.132002       3        1   2.781511
3     -0.104223 -0.138504       4        1  11.599562
4      0.320040  0.015823       5        1   2.664208
5     -0.069898  0.207534       6        1  18.510853
6     -0.014816  0.147202       7        1   6.838125
7     -0.137588 -0.124996       8        1  10.556173
8     -0.105203 -0.054606       9        1  14.064396
9     -0.123244  0.081689      10        1  13.681155, topic_info=     Category         Freq                   Term        Total  loglift  \
481   Default  4849.000000             transition  4849.000000  30.0000   
572   Default  4038.000000               produits  4038.000000  29.0000   
87    Default  5507.000000             écologique  5507.000000  28.0000   
260   Default  3020.000000                énergie  3020.000000  27.0000   
482   Default  2981.000000  transition_écologique  2981.000000  26.0000   
696   Default  2485.000000              nucléaire  2485.000000  25.0000   
486   Default  2236.000000               énergies  2236.000000  24.0000   
445   Default  2450.000000                voiture  2450.000000  23.0000   
164   Default  2001.000000              véhicules  2001.000000  22.0000   
185   Default  6121.000000                  faire  6121.000000  21.0000   
492   Default  1602.000000            agriculture  1602.000000  20.0000   
800   Default  1740.000000               voitures  1740.000000  19.0000   
715   Default  2150.000000             production  2150.000000  18.0000   
450   Default  2109.000000             électrique  2109.000000  17.0000   
327   Default  2256.000000          environnement  2256.000000  16.0000   
832   Default  1724.000000              interdire  1724.000000  15.0000   
114   Default  2470.000000            entreprises  2470.000000  14.0000   
0     Default  1205.000000              centrales  1205.000000  13.0000   
449   Default  1299.000000            électricité  1299.000000  12.0000   
873   Default   687.000000                vitesse   687.000000  11.0000   
273   Default  1100.000000                animaux  1100.000000  10.0000   
669   Default  1093.000000          renouvelables  1093.000000   9.0000   
838   Default  1556.000000              transport  1556.000000   8.0000   
446   Default  1190.000000               véhicule  1190.000000   7.0000   
485   Default  1066.000000                 villes  1066.000000   6.0000   
263   Default   994.000000              éoliennes   994.000000   5.0000   
685   Default   970.000000             pesticides   970.000000   4.0000   
862   Default  1113.000000                 diesel  1113.000000   3.0000   
1056  Default   963.000000           gouvernement   963.000000   2.0000   
71    Default   690.000000             protection   690.000000   1.0000   
...       ...          ...                    ...          ...      ...   
333   Topic10   563.428589                  impôt   678.818542   1.8028   
3605  Topic10   192.152618              bénéfices   205.434525   1.9223   
144   Topic10  1274.454224                  place  1823.771973   1.6308   
128   Topic10   368.051300        investissements   451.072479   1.7857   
1446  Topic10   449.212891               financer   583.657593   1.7273   
1066  Topic10   256.664490                 impôts   299.242188   1.8357   
527   Topic10   212.121521             financière   238.048141   1.8738   
255   Topic10   761.256836                  taxes  1188.963867   1.5433   
600   Topic10   457.074127           mettre_place   651.718628   1.6344   
114   Topic10  1265.806763            entreprises  2470.497559   1.3204   
1695  Topic10   292.515289                dépense   374.023804   1.7433   
653   Topic10   364.199036               national   516.642761   1.6395   
101   Top

Enfin, examinons les topics que le modèle reconnaît dans certains des documents individuels. Nous voyons ici comment LDA tend à attribuer une probabilité élevée à un faible nombre de sujets pour chaque document, ce qui rend ses résultats très interprétables.

In [41]:
# Nous en affichons que 8
i = 0
for (text, doc) in zip(texts[:8], docs[:8]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

***********
* doc 1   *
***********
Multiplier les centrales géothermiques
[(5, 0.27430695), (8, 0.5254362)]

***********
* doc 2   *
***********
Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040.  Avec les progrès de la communication dans ce village mondial, chaque individu, du fin fond de l’Asie au fin fond de l’Afrique, en passant par les « quartiers » et les « campagnes » de notre pays, aspire à vivre – et on ne peu l’en blâmer – comme les moins mal lotis de nos concitoyens (logement, nourriture, biens de consommation, déplacement,etc.).  Voilà la mère de tous les problèmes. Si tel est bien le cas, la solution à tous les problèmes (stabilisation de la croissance démographique, partage des richesses,

**Conclusion**

De nombreuses collections de textes non structurés ne sont pas accompagnées d'étiquettes. Les topic models (*modèles thématiques* en français) tels que le LDA sont une technique utile pour découvrir les topics les plus importants dans ces documents. **Gensim** facilite l'apprentissage sur ces sujets et **pyLDAvis** présente les résultats d'une manière visuellement attrayante. Ensemble, ils forment une puissante boîte à outils pour mieux comprendre ce qu'il y a à l'intérieur de grands ensembles de documents et pour explorer des sous-ensembles de textes connexes. Si ces résultats sont souvent déjà très révélateurs, il est également possible de les utiliser comme point de départ, par exemple pour un exercice d'étiquetage pour la classification supervisée de textes. En somme, les modèles thématiques devraient figurer dans la boîte à outils de chaque data scientist comme un moyen très rapide d'obtenir un aperçu des grandes collections de documents.